# Pneumonia Classifier Demonstration

This notebook demonstrates how to load the pretrained pneumonia classifier, perform inference on a chest X-ray image and visualize the important regions using Grad-CAM.

In [ ]:
import yaml
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

from src.model import SimpleCNN
from src.gradcam import GradCAM

# load config
with open('src/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('outputs/checkpoints/best_model.pth', map_location=device))
model.eval()

In [ ]:
# Choose a sample X-ray image
img_path = 'data/raw/chest_xray/test/NORMAL/IM-0001-0001.jpeg'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

image = Image.open(img_path).convert('L')
input_tensor = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(input_tensor)
    probs = torch.softmax(output, dim=1)
    pred = probs.argmax(dim=1).item()

classes = ['NORMAL', 'PNEUMONIA']
print('Predicted class:', classes[pred])

In [ ]:
gradcam = GradCAM(model, model.conv3)
cam = gradcam.generate(input_tensor)

img = np.array(image.resize((224, 224)))
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
overlay = cv2.addWeighted(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), 0.6, heatmap, 0.4, 0)

plt.imshow(overlay)
plt.axis('off')
plt.show()